In [1]:
#libraries importing
import pandas as pd

In [2]:
#reading the NYT_API data
NYT_df=pd.read_csv("output/best_book_list.csv")
NYT_df.head()

,rank,primary_isbn10,primary_isbn13,publisher,title,author,category,pub_date
0,1,0525559477,9780525559474,Viking,THE MIDNIGHT LIBRARY,Matt Haig,Hardcover Fiction,current
1,2,0593356152,9780593356159,Ballantine,THE MAID,Nita Prose,Hardcover Fiction,current
2,3,0735222355,9780735222359,Viking,THE LINCOLN HIGHWAY,Amor Towles,Hardcover Fiction,current
3,4,0316499773,9780316499774,"Little, Brown",THE HORSEWOMAN,James Patterson and Mike Lupica,Hardcover Fiction,current
4,5,0593185412,9780593185414,Dutton,ONE STEP TOO FAR,Lisa Gardner,Hardcover Fiction,current


In [3]:
#checking columns and null values
NYT_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820 entries, 0 to 1819
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   rank            1820 non-null   int64 
 1   primary_isbn10  1523 non-null   object
 2   primary_isbn13  1820 non-null   object
 3   publisher       1820 non-null   object
 4   title           1820 non-null   object
 5   author          1820 non-null   object
 6   category        1820 non-null   object
 7   pub_date        1820 non-null   object
dtypes: int64(1), object(7)
memory usage: 113.9+ KB


In [4]:
#dropping column with null values
NYT_df.drop('primary_isbn10',axis=1,inplace=True)

In [5]:
#checking duplicated rows
NYT_df.duplicated().sum()

0

In [6]:
NYT_df["category"].value_counts()

Combined Print Nonfiction    280
Combined Print Fiction       280
E-Book Nonfiction            210
E-Book Fiction               210
Audio Fiction                210
Audio Nonfiction             210
Hardcover Fiction            210
Hardcover Nonfiction         210
Name: category, dtype: int64

In [7]:
# checking the unique number of title column
print(f'The unique number of title column: {NYT_df["title"].nunique()}')
NYT_df["title"].value_counts().to_frame()

The unique number of title column: 586


,title
THE MIDNIGHT LIBRARY,26
THE STORYTELLER,22
GREENLIGHTS,20
BORN A CRIME,19
THE LAST THING HE TOLD ME,19
...,...
I'LL SEE YOU AGAIN,1
THE ALCHEMIST,1
BUFFERING,1
THE SENTINEL,1


In [13]:
# changing the pub_date column to datetime format and sorting dataframe based on the pub_date
NYT_df.loc[NYT_df['pub_date']=='current','pub_date']="2022-02-02"
NYT_df['pub_date'] = pd.to_datetime(NYT_df['pub_date'])
NYT_df=NYT_df.sort_values(by='pub_date',ascending=False)

In [14]:
#drop duplicated rows with the same columns of 'primary_isbn13','publisher','title','author','category':
NYT_df.drop_duplicates(subset=['primary_isbn13','publisher','title','author','category'],inplace=True)

In [52]:
#NYT_df.loc[NYT_df['title']=='THE STORYTELLER']

In [17]:
NYT_df['category'].value_counts()

E-Book Fiction               116
Combined Print Fiction       111
Audio Fiction                101
Combined Print Nonfiction     87
Audio Nonfiction              84
E-Book Nonfiction             75
Hardcover Nonfiction          53
Hardcover Fiction             52
Name: category, dtype: int64

In [19]:
NYT_df.to_csv("output/clean_NYT.csv",index=False)

Obtaining Information from Rainforest API

In [30]:
#importing dependencies
import requests
import time
from pprint import pprint

# Import API key
from config import api_key

In [64]:
#reading in the NYT csv file
nyt_df = pd.read_csv('output/clean_NYT.csv')

nyt_df.head()

,rank,primary_isbn13,publisher,title,author,category,pub_date
0,1,9780525559474,Viking,THE MIDNIGHT LIBRARY,Matt Haig,Hardcover Fiction,2022-02-02
1,8,9780310330912,Zondervan,AMERICA THE BEAUTIFUL,Ben C Carson and Candy Carson,Combined Print Nonfiction,2022-02-02
2,20,9781400064168,Random House,UNBROKEN,Laura Hillenbrand,Combined Print Nonfiction,2022-02-02
3,19,9781451674774,Gallery,I'LL SEE YOU AGAIN,Jackie Hance with Janice Kaplan,Combined Print Nonfiction,2022-02-02
4,18,9780374533557,"Farrar, Straus & Giroux","THINKING, FAST AND SLOW",Daniel Kahneman,Combined Print Nonfiction,2022-02-02


In [65]:
#setting up additional columns to hold information
nyt_df['Amazon Product ID'] = ""
nyt_df['Amazon Rating'] = ""
nyt_df['Amazon Rating Total'] = ""
nyt_df['Amazon Price'] = ""

nyt_df.head()

,rank,primary_isbn13,publisher,title,author,category,pub_date,Amazon Product ID,Amazon Rating,Amazon Rating Total,Amazon Price
0,1,9780525559474,Viking,THE MIDNIGHT LIBRARY,Matt Haig,Hardcover Fiction,2022-02-02,,,,
1,8,9780310330912,Zondervan,AMERICA THE BEAUTIFUL,Ben C Carson and Candy Carson,Combined Print Nonfiction,2022-02-02,,,,
2,20,9781400064168,Random House,UNBROKEN,Laura Hillenbrand,Combined Print Nonfiction,2022-02-02,,,,
3,19,9781451674774,Gallery,I'LL SEE YOU AGAIN,Jackie Hance with Janice Kaplan,Combined Print Nonfiction,2022-02-02,,,,
4,18,9780374533557,"Farrar, Straus & Giroux","THINKING, FAST AND SLOW",Daniel Kahneman,Combined Print Nonfiction,2022-02-02,,,,


In [126]:
#declaring the parameters
params = {
    'api_key': api_key,
    'type': 'search',
    'amazon_domain': 'amazon.com'
    }

#using iterrows to iterate throguh pandas dataframe
for index, row in nyt_df.iterrows():
    #obtaining the ISBN of the books
    isbn = row["primary_isbn13"]
        
    #adding ISBN search_term to params
    params['search_term'] = isbn    

    #making the url for requests 
    print(f"Retrieving Results for {index} : {isbn}.")
    response = requests.get('https://api.rainforestapi.com/request', params)
    
    #converting response into json
    response=response.json()

    #try to grab the following information, if available
    try:
        nyt_df.loc[index, 'Amazon Product ID'] = response["search_results"][0]["asin"]
        nyt_df.loc[index, 'Amazon Rating'] = response["search_results"][0]["rating"]
        nyt_df.loc[index, 'Amazon Rating Total'] = response["search_results"][0]["ratings_total"]
        nyt_df.loc[index, 'Amazon Price'] = response["search_results"][0]["prices"][0]["value"]
        
        #including a condition for handling books that are not listed in Amazon
    except (KeyError, IndexError):
        print(f"Title not found. Skipping...")

Retrieving Results for 0 : 9780525559474.
Retrieving Results for 1 : 9780310330912.
Retrieving Results for 2 : 9781400064168.
Retrieving Results for 3 : 9781451674774.
Retrieving Results for 4 : 9780374533557.
Retrieving Results for 5 : 9781476703541.
Retrieving Results for 6 : 9780393081572.
Retrieving Results for 7 : 9781594204807.
Retrieving Results for 8 : 9780307957139.
Retrieving Results for 9 : 9780307352156.
Retrieving Results for 10 : 9781401943417.
Retrieving Results for 11 : 9780062238863.
Retrieving Results for 12 : 9781476706412.
Retrieving Results for 13 : 9781568586717.
Retrieving Results for 14 : 9781476711874.
Retrieving Results for 15 : 9780593230572.
Retrieving Results for 16 : 9781250024978.
Retrieving Results for 17 : 9780307476074.
Retrieving Results for 18 : 9781594204210.
Retrieving Results for 19 : 9781451695199.
Retrieving Results for 20 : 9780385349949.
Retrieving Results for 21 : 9780316154697.
Retrieving Results for 22 : 9781594488399.
Retrieving Results fo

Retrieving Results for 165 : 9780063082199.
Retrieving Results for 166 : 9781984859501.
Retrieving Results for 167 : 9781637582350.
Retrieving Results for 168 : 9780063112193.
Retrieving Results for 169 : 9780593186329.
Retrieving Results for 170 : 9781250272706.
Retrieving Results for 171 : 9780316332910.
Title not found. Skipping...
Retrieving Results for 172 : 9781982177423.
Retrieving Results for 173 : 9781984821553.
Retrieving Results for 174 : 9780063098206.
Retrieving Results for 175 : 9780593188163.
Retrieving Results for 176 : 9781984818508.
Retrieving Results for 177 : 9781982154875.
Retrieving Results for 178 : 9780593419649.
Retrieving Results for 179 : 9780593300015.
Title not found. Skipping...
Retrieving Results for 180 : 9780316535861.
Retrieving Results for 181 : 9780593241318.
Retrieving Results for 182 : DBKADBL053924.
Title not found. Skipping...
Retrieving Results for 183 : 9780593460610.
Title not found. Skipping...
Retrieving Results for 184 : 9780593458822.
Titl

Title not found. Skipping...
Retrieving Results for 293 : 9780593396889.
Title not found. Skipping...
Retrieving Results for 294 : 9781508267744.
Title not found. Skipping...
Retrieving Results for 295 : DBKADBL053923.
Title not found. Skipping...
Retrieving Results for 296 : 9781543661262.
Title not found. Skipping...
Retrieving Results for 297 : 9781797121758.
Title not found. Skipping...
Retrieving Results for 298 : 9781508251200.
Title not found. Skipping...
Retrieving Results for 299 : 9780593348208.
Title not found. Skipping...
Retrieving Results for 300 : 9781797117058.
Title not found. Skipping...
Retrieving Results for 301 : 9781797113005.
Title not found. Skipping...
Retrieving Results for 302 : 9781250752291.
Title not found. Skipping...
Retrieving Results for 303 : 9780593340103.
Title not found. Skipping...
Retrieving Results for 304 : 9781250790934.
Title not found. Skipping...
Retrieving Results for 305 : 9781797119205.
Title not found. Skipping...
Retrieving Results for

Retrieving Results for 432 : 9781939996466.
Retrieving Results for 433 : 9780307589385.
Retrieving Results for 434 : 9780698188365.
Retrieving Results for 435 : 9780062415820.
Retrieving Results for 436 : 9780698163454.
Retrieving Results for 437 : 9780062316103.
Retrieving Results for 438 : 9780698185395.
Retrieving Results for 439 : 9780316387859.
Retrieving Results for 440 : A00B01EIJC2TK.
Title not found. Skipping...
Retrieving Results for 441 : 9780062320247.
Retrieving Results for 442 : 9781681196336.
Retrieving Results for 443 : 9781250022127.
Retrieving Results for 444 : 9780997552942.
Retrieving Results for 445 : 9781942832034.
Retrieving Results for 446 : 9781250132932.
Retrieving Results for 447 : 9780062436726.
Retrieving Results for 448 : 9780399575525.
Retrieving Results for 449 : 9781501108600.
Retrieving Results for 450 : A00B01M23RUSL.
Title not found. Skipping...
Retrieving Results for 451 : 9780307889805.
Retrieving Results for 452 : A00B01KKOTNBW.
Title not found. S

Retrieving Results for 613 : 9781594205347.
Retrieving Results for 614 : 9780316017930.
Retrieving Results for 615 : 9781420118537.
Retrieving Results for 616 : 9780547928173.
Retrieving Results for 617 : 9781401324797.
Retrieving Results for 618 : 9780812993929.
Retrieving Results for 619 : 9780061967559.
Retrieving Results for 620 : 9780446571821.
Retrieving Results for 621 : 9780441020010.
Retrieving Results for 622 : 9780399158827.
Retrieving Results for 623 : 9781451647037.
Retrieving Results for 624 : 9780786963645.
Retrieving Results for 625 : 9781455523559.
Retrieving Results for 626 : 9781414319704.
Retrieving Results for 627 : 9780062078117.
Retrieving Results for 628 : 9780758280343.
Retrieving Results for 629 : 9780670026715.
Retrieving Results for 630 : 9781400067558.
Retrieving Results for 631 : 9780307700667.
Retrieving Results for 632 : 9780393087727.
Retrieving Results for 633 : 9780307408853.
Retrieving Results for 634 : 9780451233967.
Retrieving Results for 635 : 978

In [152]:
#viewing the updated dataframe
amazon_df = nyt_df

amazon_df.head(50)

,rank,primary_isbn13,publisher,title,author,category,pub_date,Amazon Product ID,Amazon Rating,Amazon Rating Total,Amazon Price,Available Estimate,Monthly Sales Estimate,Weekly Sales Estimate,Amazon Bestseller Rank,Format,Reviews Total
0,1,9780525559474,Viking,THE MIDNIGHT LIBRARY,Matt Haig,Hardcover Fiction,2022-02-02,0525559477,4.3,132597.0,13.29,,,,,,
1,8,9780310330912,Zondervan,AMERICA THE BEAUTIFUL,Ben C Carson and Candy Carson,Combined Print Nonfiction,2022-02-02,0310330912,4.8,3797.0,11.98,,,,,,
2,20,9781400064168,Random House,UNBROKEN,Laura Hillenbrand,Combined Print Nonfiction,2022-02-02,B004CJN7TG,4.8,32696.0,0.00,,,,,,
3,19,9781451674774,Gallery,I'LL SEE YOU AGAIN,Jackie Hance with Janice Kaplan,Combined Print Nonfiction,2022-02-02,B00C1LBXB6,4.7,711.0,0.00,,,,,,
4,18,9780374533557,"Farrar, Straus & Giroux","THINKING, FAST AND SLOW",Daniel Kahneman,Combined Print Nonfiction,2022-02-02,0374533555,4.6,22638.0,11.29,,,,,,
5,17,9781476703541,Howard,DUCK COMMANDERS,Willie and Korie Robertson with Mark Schlabach,Combined Print Nonfiction,2022-02-02,B009NOT8N4,4.8,2245.0,0.00,,,,,,
6,16,9780393081572,Norton,GULP,Mary Roach,Combined Print Nonfiction,2022-02-02,B00BZ5QPKI,4.5,1653.0,0.00,,,,,,
7,15,9781594204807,Penguin Press,WAY OF THE KNIFE,Mark Mazzetti,Combined Print Nonfiction,2022-02-02,014312501X,4.4,508.0,13.99,,,,,,
8,14,9780307957139,Knopf,THE NEW DIGITAL AGE,Eric Schmidt and Jared Cohen,Combined Print Nonfiction,2022-02-02,0307957136,3.9,327.0,14.95,,,,,,
9,13,9780307352156,Broadway,QUIET,Susan Cain,Combined Print Nonfiction,2022-02-02,0307352153,4.6,16472.0,7.73,,,,,,


In [153]:
amazon_df2 = amazon_df.loc[:, ["rank", "primary_isbn13", "publisher", "title", "category", "pub_date", 
                               "Amazon Product ID", "Amazon Price"]]

amazon_df2

,rank,primary_isbn13,publisher,title,category,pub_date,Amazon Product ID,Amazon Price
0,1,9780525559474,Viking,THE MIDNIGHT LIBRARY,Hardcover Fiction,2022-02-02,0525559477,13.29
1,8,9780310330912,Zondervan,AMERICA THE BEAUTIFUL,Combined Print Nonfiction,2022-02-02,0310330912,11.98
2,20,9781400064168,Random House,UNBROKEN,Combined Print Nonfiction,2022-02-02,B004CJN7TG,0.00
3,19,9781451674774,Gallery,I'LL SEE YOU AGAIN,Combined Print Nonfiction,2022-02-02,B00C1LBXB6,0.00
4,18,9780374533557,"Farrar, Straus & Giroux","THINKING, FAST AND SLOW",Combined Print Nonfiction,2022-02-02,0374533555,11.29
...,...,...,...,...,...,...,...,...
674,7,9780061957918,HarperCollins Publishers,EVER AFTER,Combined Print Fiction,2013-02-10,B00A2ZIP0A,0.00
675,15,9781451640205,Simon & Schuster,THE THIRD BULLET,Combined Print Fiction,2013-02-10,B006VFZPBW,8.99
676,16,9780446571784,Grand Central,PRIVATE: #1 SUSPECT,Combined Print Fiction,2013-02-10,0446571784,15.99
677,17,9780385535144,Knopf Doubleday Publishing,THE RACKETEER,Combined Print Fiction,2013-02-10,B009UX6DYA,0.00


In [156]:
#cleaning the dataframe
#dropping any books without Amazon data
clean_amazon_df=amazon_df2.dropna(axis=0, how="any")

#viewing the cleaned dataframe
clean_amazon_df.head(20)
clean_amazon_df.count()

rank                 473
primary_isbn13       473
publisher            473
title                473
category             473
pub_date             473
Amazon Product ID    473
Amazon Price         473
dtype: int64

In [157]:
#writing the file to a csv
clean_amazon_df.to_csv("output/combined_book.csv",index=False)